In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [3]:
file = 'data/Hz31.txt'
data = pd.read_csv(file, names=['z', 'hz', 'err'], sep = " " )

# odds = [x for x in range(len(data)) if x%2 ==0]
# even = [x for x in range(len(data)) if x%2 ==1]
# new_order = np.array(even+odds)

a = np.array([25, 3, 2, 27, 14, 30, 21, 9, 8, 18, 20, 6, 22, 19, 5, 17, 4, 13, 0, 24, 29, 7, 15, 1, 12, 23, 16, 28, 10, 26, 11])
data = data.values[a]


X = data[:,0]
y = data[:,1:]

scalerx = StandardScaler()
# scalerz = MinMaxScaler(feature_range=(-1,1))
# fit scaler on data
scalerx.fit(X.reshape(-1,1))
# apply transform
X = scalerx.transform(X.reshape(-1,1))


# scalery = MinMaxScaler(feature_range=(0,1))
# scalery.fit(y)
# y = scalery.transform(y)

split = 0.8
ntrain = int(split * len(X))
indx = [ntrain]
X_train, X_test = np.split(X, indx)
y_train, y_test = np.split(y, indx)


In [4]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0.01,
                                   patience=10,
                                   restore_best_weights=True)
                                   ]

n_cols = 1

In [5]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([100, 200]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 80, 100]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([30, 50]))
HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([4, 8, 16, 32]))

In [6]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [7]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [8]:
def autoencoder(input_z, hparams):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    efirst = Dense(hparams[HP_NUM_UNITS1], activation='relu', input_shape=(1,))(input_z)
    ehidden = Dense(hparams[HP_NUM_UNITS2], activation='relu')(efirst)
    elast = Dense(hparams[HP_NUM_UNITS3], activation='relu')(ehidden)
    coded = Dense(hparams[HP_NUM_UNITS4], activation='relu')(elast)
    #decoder 
    dfirst = Dense(hparams[HP_NUM_UNITS3], activation='relu')(coded)
    dhidden = Dense(hparams[HP_NUM_UNITS2], activation='relu')(dfirst)
    dlast = Dense(hparams[HP_NUM_UNITS1], activation='relu')(dhidden)
    decoded = Dense(2, activation='relu')(dlast)
    
    return decoded

In [9]:
epochs = 200
input_z = tf.keras.layers.Input(shape = (1,))

In [10]:
def train_test_model(hparams):
    model = Model(input_z, autoencoder(input_z, hparams))  
                              
    model.compile(
#             optimizer=hparams[HP_OPTIMIZER],
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(X_train, y_train, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks) 
    _, loss = model.evaluate(X_test, y_test)


    return loss
    

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
#     with tf.summary.FileWriter(run_dir):
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [12]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
                for num_units4 in HP_NUM_UNITS4.domain.values:
#       for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#                     for optimizer in HP_OPTIMIZER.domain.values:
                    hparams = {
                      HP_NUM_UNITS1: num_units1,
                      HP_NUM_UNITS2: num_units2,
                      HP_NUM_UNITS3: num_units3,
                      HP_NUM_UNITS4: num_units4,
                    #                       HP_DROPOUT: dropout_rate,
#                       HP_OPTIMIZER: optimizer,
                      HP_BATCHSIZE: batch_size
                    }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    run('logs/hparam_tuning/' + run_name, hparams)
                    session_num += 1

--- Starting trial: run-0
{'num_units1': 100, 'num_units2': 50, 'num_units3': 30, 'num_units4': 2, 'batch_size': 4}
Epoch 1/200
6/6 [==============================] - 0s 2ms/step - loss: 6723.7661 - mean_squared_error: 6723.7661
Epoch 2/200
6/6 [==============================] - 0s 1ms/step - loss: 6723.7671 - mean_squared_error: 6723.7671
Epoch 3/200
6/6 [==============================] - 0s 1ms/step - loss: 6723.7661 - mean_squared_error: 6723.7661
Epoch 4/200
6/6 [==============================] - 0s 1ms/step - loss: 6723.7661 - mean_squared_error: 6723.7661
Epoch 5/200
6/6 [==============================] - 0s 1ms/step - loss: 6723.7671 - mean_squared_error: 6723.7671
Epoch 6/200
6/6 [==============================] - 0s 1ms/step - loss: 6723.7671 - mean_squared_error: 6723.7671
Epoch 7/200
6/6 [==============================] - 0s 1ms/step - loss: 6723.7671 - mean_squared_error: 6723.7671
Epoch 8/200
6/6 [==============================] - 0s 1ms/step - loss: 6723.7661 - mean_squar

6/6 [==============================] - 0s 868us/step - loss: 182.2198 - mean_squared_error: 182.2198
Epoch 60/200
6/6 [==============================] - 0s 1ms/step - loss: 176.6507 - mean_squared_error: 176.6507
Epoch 61/200
6/6 [==============================] - 0s 953us/step - loss: 176.6796 - mean_squared_error: 176.6796
Epoch 62/200
6/6 [==============================] - 0s 901us/step - loss: 177.8985 - mean_squared_error: 177.8985
Epoch 63/200
6/6 [==============================] - 0s 903us/step - loss: 180.8774 - mean_squared_error: 180.8774
Epoch 64/200
6/6 [==============================] - 0s 828us/step - loss: 178.1663 - mean_squared_error: 178.1663
Epoch 65/200
6/6 [==============================] - 0s 960us/step - loss: 174.1574 - mean_squared_error: 174.1574
Epoch 66/200
6/6 [==============================] - 0s 895us/step - loss: 175.7103 - mean_squared_error: 175.7103
Epoch 67/200
6/6 [==============================] - 0s 1ms/step - loss: 172.3256 - mean_squared_error: 

6/6 [==============================] - 0s 1ms/step - loss: 6452.8296 - mean_squared_error: 6452.8296
Epoch 44/200
6/6 [==============================] - 0s 1ms/step - loss: 6454.9604 - mean_squared_error: 6454.9604
Epoch 45/200
6/6 [==============================] - 0s 1ms/step - loss: 6455.9453 - mean_squared_error: 6455.9453
Epoch 46/200
6/6 [==============================] - 0s 1ms/step - loss: 6452.1133 - mean_squared_error: 6452.1133
Epoch 47/200
6/6 [==============================] - 0s 1ms/step - loss: 6451.8086 - mean_squared_error: 6451.8086
Epoch 48/200
6/6 [==============================] - 0s 1ms/step - loss: 6451.9087 - mean_squared_error: 6451.9087
Epoch 49/200
6/6 [==============================] - 0s 1ms/step - loss: 6451.6875 - mean_squared_error: 6451.6875
Epoch 50/200
6/6 [==============================] - 0s 1ms/step - loss: 6451.4985 - mean_squared_error: 6451.4985
Epoch 51/200
6/6 [==============================] - 0s 1ms/step - loss: 6451.0991 - mean_squared_erro

6/6 [==============================] - 0s 1ms/step - loss: 6428.5430 - mean_squared_error: 6428.5430
Epoch 115/200
6/6 [==============================] - 0s 2ms/step - loss: 6426.6938 - mean_squared_error: 6426.6938
Epoch 116/200
6/6 [==============================] - 0s 1ms/step - loss: 6427.9878 - mean_squared_error: 6427.9878
Epoch 117/200
6/6 [==============================] - 0s 2ms/step - loss: 6426.2397 - mean_squared_error: 6426.2397
Epoch 118/200
6/6 [==============================] - 0s 1ms/step - loss: 6428.8130 - mean_squared_error: 6428.8130
Epoch 119/200
6/6 [==============================] - 0s 935us/step - loss: 6430.6450 - mean_squared_error: 6430.6450
Epoch 120/200
6/6 [==============================] - 0s 1ms/step - loss: 6424.9844 - mean_squared_error: 6424.9844
Epoch 121/200
6/6 [==============================] - 0s 887us/step - loss: 6425.0195 - mean_squared_error: 6425.0195
Epoch 122/200
6/6 [==============================] - 0s 938us/step - loss: 6425.1372 - mea

6/6 [==============================] - 0s 1ms/step - loss: 6411.7876 - mean_squared_error: 6411.7876
Epoch 186/200
6/6 [==============================] - 0s 1ms/step - loss: 6414.2563 - mean_squared_error: 6414.2563
Epoch 187/200
6/6 [==============================] - 0s 2ms/step - loss: 6410.3164 - mean_squared_error: 6410.3164
Epoch 188/200
6/6 [==============================] - 0s 2ms/step - loss: 6409.9526 - mean_squared_error: 6409.9526
Epoch 189/200
6/6 [==============================] - 0s 1ms/step - loss: 6410.9829 - mean_squared_error: 6410.9829
Epoch 190/200
6/6 [==============================] - 0s 1ms/step - loss: 6409.9019 - mean_squared_error: 6409.9019
Epoch 191/200
6/6 [==============================] - 0s 1ms/step - loss: 6414.1089 - mean_squared_error: 6414.1089
Epoch 192/200
6/6 [==============================] - 0s 2ms/step - loss: 6409.9233 - mean_squared_error: 6409.9233
Epoch 193/200
6/6 [==============================] - 0s 2ms/step - loss: 6409.7954 - mean_squa

...

1/1 [==============================] - 0s 4ms/step - loss: 591.0002 - mean_squared_error: 591.0002
Epoch 78/200
1/1 [==============================] - 0s 4ms/step - loss: 580.8928 - mean_squared_error: 580.8928
Epoch 79/200
1/1 [==============================] - 0s 4ms/step - loss: 573.3932 - mean_squared_error: 573.3932
Epoch 80/200
1/1 [==============================] - 0s 4ms/step - loss: 567.9385 - mean_squared_error: 567.9385
Epoch 81/200
1/1 [==============================] - 0s 5ms/step - loss: 563.7364 - mean_squared_error: 563.7364
Epoch 82/200
1/1 [==============================] - 0s 4ms/step - loss: 559.9877 - mean_squared_error: 559.9877
Epoch 83/200
1/1 [==============================] - 0s 4ms/step - loss: 556.1290 - mean_squared_error: 556.1290
Epoch 84/200
1/1 [==============================] - 0s 3ms/step - loss: 551.9286 - mean_squared_error: 551.9286
Epoch 85/200
1/1 [==============================] - 0s 4ms/step - loss: 547.3978 - mean_squared_error: 547.3978
Epoch

1/1 [==============================] - 0s 3ms/step - loss: 446.4993 - mean_squared_error: 446.4993
Epoch 150/200
1/1 [==============================] - 0s 3ms/step - loss: 446.1566 - mean_squared_error: 446.1566
Epoch 151/200
1/1 [==============================] - 0s 3ms/step - loss: 445.8225 - mean_squared_error: 445.8225
Epoch 152/200
1/1 [==============================] - 0s 3ms/step - loss: 445.4966 - mean_squared_error: 445.4966
Epoch 153/200
1/1 [==============================] - 0s 3ms/step - loss: 445.1792 - mean_squared_error: 445.1792
Epoch 154/200
1/1 [==============================] - 0s 3ms/step - loss: 444.8711 - mean_squared_error: 444.8711
Epoch 155/200
1/1 [==============================] - 0s 3ms/step - loss: 444.5723 - mean_squared_error: 444.5723
Epoch 156/200
1/1 [==============================] - 0s 3ms/step - loss: 444.2814 - mean_squared_error: 444.2814
Epoch 157/200
1/1 [==============================] - 0s 3ms/step - loss: 443.9984 - mean_squared_error: 443.99

Epoch 13/200
1/1 [==============================] - 0s 3ms/step - loss: 6605.2734 - mean_squared_error: 6605.2734
Epoch 14/200
1/1 [==============================] - 0s 3ms/step - loss: 6578.4023 - mean_squared_error: 6578.4023
Epoch 15/200
1/1 [==============================] - 0s 3ms/step - loss: 6546.4824 - mean_squared_error: 6546.4824
Epoch 16/200
1/1 [==============================] - 0s 2ms/step - loss: 6508.3188 - mean_squared_error: 6508.3188
Epoch 17/200
1/1 [==============================] - 0s 3ms/step - loss: 6462.8813 - mean_squared_error: 6462.8813
Epoch 18/200
1/1 [==============================] - 0s 3ms/step - loss: 6408.8491 - mean_squared_error: 6408.8491
Epoch 19/200
1/1 [==============================] - 0s 3ms/step - loss: 6344.6235 - mean_squared_error: 6344.6235
Epoch 20/200
1/1 [==============================] - 0s 2ms/step - loss: 6268.8882 - mean_squared_error: 6268.8882
Epoch 21/200
1/1 [==============================] - 0s 2ms/step - loss: 6179.7349 - mean

1/1 [==============================] - 0s 4ms/step - loss: 263.6781 - mean_squared_error: 263.6781
Epoch 86/200
1/1 [==============================] - 0s 4ms/step - loss: 259.7468 - mean_squared_error: 259.7468
Epoch 87/200
1/1 [==============================] - 0s 4ms/step - loss: 255.1921 - mean_squared_error: 255.1921
Epoch 88/200
1/1 [==============================] - 0s 5ms/step - loss: 250.1388 - mean_squared_error: 250.1388
Epoch 89/200
1/1 [==============================] - 0s 4ms/step - loss: 244.9373 - mean_squared_error: 244.9373
Epoch 90/200
1/1 [==============================] - 0s 4ms/step - loss: 240.0119 - mean_squared_error: 240.0119
Epoch 91/200
1/1 [==============================] - 0s 5ms/step - loss: 235.6828 - mean_squared_error: 235.6828
Epoch 92/200
1/1 [==============================] - 0s 4ms/step - loss: 232.1087 - mean_squared_error: 232.1087
Epoch 93/200
1/1 [==============================] - 0s 3ms/step - loss: 229.1995 - mean_squared_error: 229.1995
Epoch

1/1 [==============================] - 0s 4ms/step - loss: 170.7563 - mean_squared_error: 170.7563
Epoch 158/200
1/1 [==============================] - 0s 6ms/step - loss: 170.6042 - mean_squared_error: 170.6042
Epoch 159/200
1/1 [==============================] - 0s 4ms/step - loss: 170.4560 - mean_squared_error: 170.4560
Epoch 160/200
1/1 [==============================] - 0s 4ms/step - loss: 170.3124 - mean_squared_error: 170.3124
Epoch 161/200
1/1 [==============================] - 0s 4ms/step - loss: 170.1714 - mean_squared_error: 170.1714
Epoch 162/200
1/1 [==============================] - 0s 3ms/step - loss: 170.0352 - mean_squared_error: 170.0352
Epoch 163/200
1/1 [==============================] - 0s 3ms/step - loss: 169.9026 - mean_squared_error: 169.9026
Epoch 164/200
1/1 [==============================] - 0s 2ms/step - loss: 169.7731 - mean_squared_error: 169.7731
Epoch 165/200
1/1 [==============================] - 0s 3ms/step - loss: 169.6549 - mean_squared_error: 169.65

In [13]:
!kill 2609
%tensorboard --logdir logs/hparam_tuning

/bin/sh: 1: kill: No such process



Reusing TensorBoard on port 6006 (pid 7893), started 1:08:47 ago. (Use '!kill 7893' to kill it.)